# IBM Coursera Capstone 2020

### Introduction/Business Problem

Downtown Jacksonville is divided into eight zoning districts that allow and encourage particular uses - industrial, retail, residential, etc. I will use Foursquare location data to compare the types and amounts of businesses in each of these zoning districts. This would be a useful analysis for Jacksonville's Downtown Investment Authority as it decides how to divide funds for incentives and is it looks to see which businesses downtown need the most support after the Covid-19 pandemic.

In [1]:
from IPython.display import HTML
HTML("""
<div class='tableauPlaceholder' id='viz1590365560426' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Do&#47;DowntownJacksonvilleZoningOverlay&#47;DowntownJacksonvilleZoningOverlay&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='DowntownJacksonvilleZoningOverlay&#47;DowntownJacksonvilleZoningOverlay' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Do&#47;DowntownJacksonvilleZoningOverlay&#47;DowntownJacksonvilleZoningOverlay&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1590365560426');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>""")

### Data

I will be using Jacksonville's downtown zoning overlay to cluster businesses in Foursquare's database. 

First, I will upload a CSV file of the latitude and longitudes of the properties in each zoning district, which I configured from the Duval County Property Appraiser's database and a map of the downtown zoning overlay. There are 1,007 addresses in this file.

Then, I will query Foursquare to find the venue closest to each of these locations. I will then group the results by the downtown zoning overlays from the original dataset.


#### Step 1
Import libraries.

In [2]:
#!conda install -c conda-forge geopy --yes # uncomment this line on first use
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line on first use
import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np
import pandas as pd

#### Step 2
Import the file with locations from the Duval County Property Appraiser.

*Note: Foursquare only allows 950 free get requests a day, so since I have 1,007 locations with wich to query, I made requests over two days and saved the first day's results to a separate CSV, which I appended to the second day's dataframe.

In [3]:
#open file of the latitudes and longitudes of each downtown parcel
fh = "C://Users//Robin//Documents//DowntownOverlay.csv"
df = pd.read_csv(fh)
df.drop(columns = ['Unnamed: 4','Unnamed: 5','Unnamed: 6'], inplace=True)

#drop null values
df.dropna(inplace=True)

#remove latitudes and longitudes that are outside the downtown boundaries
dt_jax = df[~df['Address'].str.startswith("0")]
dt_jax = dt_jax[~dt_jax['Address'].str.startswith(",")]
lst = ['100 MYRTLE AVE, JACKSONVILLE, 32204','757 UNION ST W, JACKSONVILLE, 32202','816 UNION ST W, JACKSONVILLE, 32202',
       '958 STATE ST W, JACKSONVILLE, 32204','220 MYRTLE AVE S, JACKSONVILLE, 32204','903 UNION ST W, JACKSONVILLE, 32204',
      '961 UNION ST W, JACKSONVILLE, 32204']
dt_jax.set_index('Address',inplace=True)
dt_jax.drop(lst,inplace=True)
dt_jax.reset_index(inplace=True)

#dt_jax = dt_jax[:650]
dt_jax = dt_jax[650:]

In [4]:
dt_jax.head()

,Address,ZoningOverlayDistrict,Latitude,Longitude
650,"321 OCEAN ST N, JACKSONVILLE, 32202",Cathedral,30.328410,-81.656042
651,"105 MONROE ST E, JACKSONVILLE, 32202",Cathedral,30.328386,-81.655432
652,"120 STATE ST E, JACKSONVILLE, 32202",Cathedral,30.333441,-81.658090
653,"325 DUVAL ST E, JACKSONVILLE, 32202",Cathedral,30.328484,-81.652779
654,"101 ADAMS ST E, JACKSONVILLE, 32202",Cathedral,30.328247,-81.659005


In [8]:
#coordinates of downtown Jacksonville
latitude = 30.324282
longitude = -81.658448

# create map of all downtown Jacksonville properties
map_PropertyAppraiser = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(dt_jax['Latitude'], dt_jax['Longitude'], dt_jax['Address']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_PropertyAppraiser)  
    
map_PropertyAppraiser

#### Step 3

Make get requests to Foursquare.

*Note: insert your Foursquare credentials here

In [9]:
#define paramaters for get requests
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200526' # Foursquare API version
LIMIT = 1

Define a function to make get requests for every location in the dataframe.

In [10]:
def getNearbyVenues(names, latitudes, longitudes, districts, radius=500):
    
    venues_list=[]
    for name, lat, lng, district in zip(names, latitudes, longitudes, districts):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
       
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                district,
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:continue

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Address', 
                  'Address Latitude', 
                  'Address Longitude',
                  'Zoning District',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Make the queries using the above function.

In [11]:
# type your answer here
jax_venues = getNearbyVenues(names=dt_jax['Address'],
                            latitudes=dt_jax['Latitude'],
                            longitudes=dt_jax['Longitude'],
                            districts=dt_jax['ZoningOverlayDistrict'],
                            radius=100)

321 OCEAN ST N, JACKSONVILLE, 32202
105 MONROE ST E, JACKSONVILLE, 32202
120 STATE ST E, JACKSONVILLE, 32202
325 DUVAL ST E, JACKSONVILLE, 32202
101 ADAMS ST E, JACKSONVILLE, 32202
421 WASHINGTON ST N, JACKSONVILLE, 32202
107 DUVAL ST E, JACKSONVILLE, 32202
515 MARKET ST N, JACKSONVILLE, 32202
501 DUVAL ST E, JACKSONVILLE, 32202
256 DUVAL ST E, JACKSONVILLE, 32202
425 LIBERTY ST N, JACKSONVILLE, 32202
334 ASHLEY ST E, JACKSONVILLE, 32202
138 DUVAL ST E, JACKSONVILLE, 32202
305 WASHINGTON ST N, JACKSONVILLE, 32202
218 ASHLEY ST E, JACKSONVILLE, 32202
400 DUVAL ST E, JACKSONVILLE, 32202
601 NEWNAN ST N, JACKSONVILLE, 32202
511 WASHINGTON ST N, JACKSONVILLE, 32202
513 MARKET ST N, JACKSONVILLE, 32202
325 CHURCH ST E, JACKSONVILLE, 32202
507 MARKET ST N, JACKSONVILLE, 32202
214 ASHLEY ST E, JACKSONVILLE, 32202
101 UNION ST E, JACKSONVILLE, 32202
325 MAIN ST N, JACKSONVILLE, 32202
623 WASHINGTON ST N, JACKSONVILLE, 32202
202 WASHINGTON ST N, JACKSONVILLE, 32202
519 WASHINGTON ST N, JACKSONV

1712 BEAVER ST E, JACKSONVILLE, 32202
441 HAINES ST, JACKSONVILLE, 32202
541 TALLEYRAND AVE, JACKSONVILLE, 32202
220 TALLEYRAND AVE, JACKSONVILLE, 32202
1701 DUVAL ST E, JACKSONVILLE, 32202
1702 BEAVER ST E, JACKSONVILLE, 32202
236 BRYAN ST, JACKSONVILLE, 32202
1705 ADAMS ST E, JACKSONVILLE, 32202
1601 DUVAL ST E, JACKSONVILLE, 32202
225 TALLEYRAND AVE, JACKSONVILLE, 32202
1823 BEAVER ST E, JACKSONVILLE, 32202
410 VICTORIA ST, JACKSONVILLE, 32202
1736 BEAVER ST E, JACKSONVILLE, 32202
249 PARKER ST, JACKSONVILLE, 32202
515 TALLEYRAND AVE, JACKSONVILLE, 32202
1859 ADAMS ST E, JACKSONVILLE, 32202
243 PARKER ST, JACKSONVILLE, 32202
1836 ALBERT ST, JACKSONVILLE, 32202
1718 CHURCH ST, JACKSONVILLE, 32202
1731 BEAVER ST E, JACKSONVILLE, 32202
1740 ALBERT ST, JACKSONVILLE, 32202
1829 BEAVER ST E, JACKSONVILLE, 32202
1602 CHURCH ST, JACKSONVILLE, 32202
1718 BEAVER ST E, JACKSONVILLE, 32202
1931 BEAVER ST E, JACKSONVILLE, 32202
444 TALLEYRAND AVE, JACKSONVILLE, 32202
1559 BEAVER ST E, JACKSONVIL

In [12]:
print(jax_venues.shape)
jax_venues

(176, 8)


,Address,Address Latitude,Address Longitude,Zoning District,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"321 OCEAN ST N, JACKSONVILLE, 32202",30.328410,-81.656042,Cathedral,Burrito Gallery,30.327790,-81.656750,Burrito Place
1,"101 ADAMS ST E, JACKSONVILLE, 32202",30.328247,-81.659005,Cathedral,Chamblin's Uptown,30.328552,-81.658890,Coffee Shop
2,"107 DUVAL ST E, JACKSONVILLE, 32202",30.329243,-81.655665,Cathedral,Ritz,30.329352,-81.655376,Theater
3,"425 LIBERTY ST N, JACKSONVILLE, 32202",30.328652,-81.651559,Cathedral,No Compromise Records,30.329086,-81.651305,Record Shop
4,"334 ASHLEY ST E, JACKSONVILLE, 32202",30.329560,-81.652099,Cathedral,No Compromise Records,30.329086,-81.651305,Record Shop
...,...,...,...,...,...,...,...,...
171,"1400 PRUDENTIAL DR, JACKSONVILLE, 32207",30.316032,-81.656125,Southbank,The Wine Cellar,30.315923,-81.657123,Café
172,"1300 RIVERPLACE BLVD, JACKSONVILLE, 32207",30.318121,-81.656667,Southbank,Southern Grill,30.317727,-81.657437,Diner
173,"1451 HOME ST, JACKSONVILLE, 32207",30.315552,-81.655111,Southbank,bb's restaurant and bar,30.315882,-81.654662,American Restaurant
174,"1037 HENDRICKS AVE, JACKSONVILLE, 32207",30.315434,-81.654320,Southbank,bb's restaurant and bar,30.315882,-81.654662,American Restaurant


In [ ]:
#drop duplicate venues
jax_venues2 = jax_venues.drop_duplicates(subset =['Venue Latitude','Venue Longitude'], keep = 'first', inplace = False) 
jax_venues2.head()

In [9]:
jax_venues2.reset_index(inplace=True)
jax_venues2.drop(columns=['index'],inplace=True)
jax_venues2.head()

C:\Users\robin\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Address,Address Latitude,Address Longitude,Zoning District,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"929 PENINSULAR PL, JACKSONVILLE, 32204",30.318785,-81.679842,Brooklyn,Jimmy Johns,30.319002,-81.678994,Sandwich Place
1,"1028 ROSSELLE ST, JACKSONVILLE, 32204",30.320057,-81.680198,Brooklyn,Kanine Social,30.320710,-81.680780,Dog Run
2,"707 PENINSULAR PL, JACKSONVILLE, 32204",30.317939,-81.677455,Brooklyn,Friday Musicale,30.318300,-81.677652,Concert Hall
3,"632 CHELSEA ST, JACKSONVILLE, 32204",30.319350,-81.680175,Brooklyn,School 4 (Annie Lytle Elementary School),30.318714,-81.680491,Historic Site
4,"806 ROSSELLE ST, JACKSONVILLE, 32204",30.319310,-81.677895,Brooklyn,AJ's Barbecue,30.319334,-81.678900,BBQ Joint


In [14]:
#cluster by zoning district
clusters = []
for district in jax_venues2['Zoning District']:
    if district == 'Brooklyn':
        clusters.append(1)
    elif district == 'LaVilla':
        clusters.append(2)
    elif district == 'Church':
        clusters.append(3)
    elif district == 'Central Core':
        clusters.append(4)
    elif district == 'Cathedral':
        clusters.append(5)
    elif district == 'Sports & Entertainment':
        clusters.append(6)
    elif district == 'Working Waterfront':
        clusters.append(7)
    elif district == 'Southbank':
        clusters.append(8)
jax_venues2['Cluster'] = clusters

C:\Users\robin\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
jax_venues2.head()

,Address,Address Latitude,Address Longitude,Zoning District,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster
0,"321 OCEAN ST N, JACKSONVILLE, 32202",30.328410,-81.656042,Cathedral,Burrito Gallery,30.327790,-81.656750,Burrito Place,5
1,"101 ADAMS ST E, JACKSONVILLE, 32202",30.328247,-81.659005,Cathedral,Chamblin's Uptown,30.328552,-81.658890,Coffee Shop,5
2,"107 DUVAL ST E, JACKSONVILLE, 32202",30.329243,-81.655665,Cathedral,Ritz,30.329352,-81.655376,Theater,5
3,"425 LIBERTY ST N, JACKSONVILLE, 32202",30.328652,-81.651559,Cathedral,No Compromise Records,30.329086,-81.651305,Record Shop,5
6,"101 UNION ST E, JACKSONVILLE, 32202",30.332188,-81.654748,Cathedral,Tijuana Flats,30.332184,-81.655651,Tex-Mex Restaurant,5


Here is where I added the second day's query results to the first day's results.

In [17]:
fh2 = "C://Users//Robin//Documents//Python//jax_venues.csv"
df2 = pd.read_csv(fh2)

In [21]:
df2.drop(columns=['Unnamed: 0'],inplace=True)
df2

,Address,Address Latitude,Address Longitude,Zoning District,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster
0,"929 PENINSULAR PL, JACKSONVILLE, 32204",30.318785,-81.679842,Brooklyn,Jimmy Johns,30.319002,-81.678994,Sandwich Place,1
1,"1028 ROSSELLE ST, JACKSONVILLE, 32204",30.320057,-81.680198,Brooklyn,Kanine Social,30.320710,-81.680780,Dog Run,1
2,"707 PENINSULAR PL, JACKSONVILLE, 32204",30.317939,-81.677455,Brooklyn,Friday Musicale,30.318300,-81.677652,Concert Hall,1
3,"632 CHELSEA ST, JACKSONVILLE, 32204",30.319350,-81.680175,Brooklyn,School 4 (Annie Lytle Elementary School),30.318714,-81.680491,Historic Site,1
4,"806 ROSSELLE ST, JACKSONVILLE, 32204",30.319310,-81.677895,Brooklyn,AJ's Barbecue,30.319334,-81.678900,BBQ Joint,1
...,...,...,...,...,...,...,...,...,...
56,"109 INDEPENDENT DR E, JACKSONVILLE, 32202",30.325362,-81.660884,Central Core,Starbucks,30.325287,-81.660244,Coffee Shop,4
57,"345 FORSYTH ST E, JACKSONVILLE, 32202",30.326164,-81.652865,Central Core,Dive bar,30.325404,-81.653392,Dive Bar,4
58,"21 BAY ST E, JACKSONVILLE, 32202",30.324207,-81.647751,Central Core,The Shipyard,30.323677,-81.648208,Harbor / Marina,4
59,"420 CHURCH ST E, JACKSONVILLE, 32202",30.328502,-81.651220,Cathedral,No Compromise Records,30.329086,-81.651305,Record Shop,5


In [88]:
venues = df2.append(jax_venues2)

In [89]:
venues.tail()

,Address,Address Latitude,Address Longitude,Zoning District,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster
151,"1430 PRUDENTIAL DR, JACKSONVILLE, 32207",30.316064,-81.655775,Southbank,Extended Stay America,30.316644,-81.655489,Hotel,8
153,"1501 RIVERPLACE BLVD, JACKSONVILLE, 32207",30.318221,-81.653685,Southbank,Chart House Restaurant,30.318219,-81.653720,Seafood Restaurant,8
161,"1515 PRUDENTIAL DR, JACKSONVILLE, 32207",30.317141,-81.653036,Southbank,Lexington Hotel and Conference Center,30.317133,-81.652862,Resort,8
162,"1207 PRUDENTIAL DR, JACKSONVILLE, 32207",30.316765,-81.658100,Southbank,Treaty Oak Park,30.316478,-81.658117,Park,8
166,"1105 SAN MARCO BLVD, JACKSONVILLE, 32207",30.314291,-81.660225,Southbank,Sherwoods Liquors Inc,30.314278,-81.660302,Dive Bar,8


In [26]:
#create a map of venues, colored by zoning district
import numpy as np
# create map
latitude = 30.324282
longitude = -81.658448
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(9)
ys = [i + x + (i*x)**2 for i in range(9)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, venue, cluster in zip(venues['Venue Latitude'], venues['Venue Longitude'], venues['Zoning District'], venues['Venue'], venues['Cluster']):
    label = folium.Popup(str(venue) + ', ' + str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Step 4

Analyze the zoning districts.

In [8]:
fh2 = "C://Users//Robin//Documents//Python//full_venues.csv"
df3 = pd.read_csv(fh2)
df3.drop(columns=['Unnamed: 0'],inplace = True)
venues=df3
venues.head()

,Venue Category,Address,Address Latitude,Address Longitude,Zoning District,Venue,Venue Latitude,Venue Longitude,Cluster
0,Sandwich Place,"929 PENINSULAR PL, JACKSONVILLE, 32204",30.318785,-81.679842,Brooklyn,Jimmy Johns,30.319002,-81.678994,1
1,Dog Run,"1028 ROSSELLE ST, JACKSONVILLE, 32204",30.320057,-81.680198,Brooklyn,Kanine Social,30.320710,-81.680780,1
2,Concert Hall,"707 PENINSULAR PL, JACKSONVILLE, 32204",30.317939,-81.677455,Brooklyn,Friday Musicale,30.318300,-81.677652,1
3,Historic Site,"632 CHELSEA ST, JACKSONVILLE, 32204",30.319350,-81.680175,Brooklyn,School 4 (Annie Lytle Elementary School),30.318714,-81.680491,1
4,BBQ Joint,"806 ROSSELLE ST, JACKSONVILLE, 32204",30.319310,-81.677895,Brooklyn,AJ's Barbecue,30.319334,-81.678900,1


In [25]:
venues.groupby('Zoning District').count()

,Venue Category,Address,Address Latitude,Address Longitude,Venue,Venue Latitude,Venue Longitude,Cluster
Zoning District,,,,,,,,
Brooklyn,13,13,13,13,13,13,13,13
Cathedral,16,16,16,16,16,16,16,16
Central Core,26,26,26,26,26,26,26,26
Church,5,5,5,5,5,5,5,5
LaVilla,5,5,5,5,5,5,5,5
Southbank,16,16,16,16,16,16,16,16
Sports & Entertainment,20,20,20,20,20,20,20,20


### Methodology

I'm trying to identify downtown businesses, so let's get rid of venues that are not businesses.

In [26]:
print(venues['Venue Category'].unique())
print(len(venues['Venue Category'].unique()))

['Sandwich Place' 'Concert Hall' 'BBQ Joint' 'Smoothie Shop' 'Dive Bar'
 'Deli / Bodega' 'Gay Bar' 'Dessert Shop' 'Flower Shop'
 'Gym / Fitness Center' 'Grocery Store' 'Falafel Restaurant'
 'Burrito Place' 'Optical Shop' 'Thrift / Vintage Store' 'Food Truck'
 'Restaurant' 'Theater' 'Storage Facility' 'Gym' 'Convenience Store'
 'Coffee Shop' 'American Restaurant' 'Lounge' 'Art Gallery' 'Speakeasy'
 'Food' 'Café' 'Pub' 'Shipping Store' 'Steakhouse' 'Rental Car Location'
 'Harbor / Marina' 'Bar' 'Record Shop' 'Daycare' 'Tex-Mex Restaurant'
 'Video Store' 'Indie Theater' 'Discount Store' 'Gift Shop'
 'General Entertainment' 'Sports Bar' 'Flea Market' 'Athletics & Sports'
 'New American Restaurant' 'Brewery' 'Baseball Stadium' 'Football Stadium'
 'Breakfast Spot' 'Basketball Stadium' 'Beer Garden' 'Sushi Restaurant'
 'Fast Food Restaurant' 'History Museum' 'Diner' 'Arts & Crafts Store'
 'Planetarium' 'Indie Movie Theater' 'Hotel' 'Seafood Restaurant' 'Resort']
62


In [19]:
#lst2 = ['Dog Run','Historic Site','Tunnel','Scenic Lookout','Plaza','Light Rail Station','Platform','Airport','Park','Other Great Outdoors','Bus Station']
venues.set_index('Venue Category',inplace=True)
venues.drop('Dog Run',inplace=True)
venues.reset_index(inplace=True)
venues

,Venue Category,Address,Address Latitude,Address Longitude,Zoning District,Venue,Venue Latitude,Venue Longitude,Cluster
0,Sandwich Place,"929 PENINSULAR PL, JACKSONVILLE, 32204",30.318785,-81.679842,Brooklyn,Jimmy Johns,30.319002,-81.678994,1
1,Concert Hall,"707 PENINSULAR PL, JACKSONVILLE, 32204",30.317939,-81.677455,Brooklyn,Friday Musicale,30.318300,-81.677652,1
2,BBQ Joint,"806 ROSSELLE ST, JACKSONVILLE, 32204",30.319310,-81.677895,Brooklyn,AJ's Barbecue,30.319334,-81.678900,1
3,Smoothie Shop,"344 DELLWOOD AVE, JACKSONVILLE, 32204",30.324218,-81.679781,Brooklyn,Grabbagreen,30.324897,-81.680056,1
4,Dive Bar,"385 PARK ST, JACKSONVILLE, 32204",30.322664,-81.676484,Brooklyn,Da Tunnel,30.322930,-81.676502,1
...,...,...,...,...,...,...,...,...,...
96,Indie Movie Theater,"1655 PRUDENTIAL DR, JACKSONVILLE, 32207",30.316994,-81.651516,Southbank,Movies In The Park @ Wyndham,30.317572,-81.651527,8
97,Hotel,"1430 PRUDENTIAL DR, JACKSONVILLE, 32207",30.316064,-81.655775,Southbank,Extended Stay America,30.316644,-81.655489,8
98,Seafood Restaurant,"1501 RIVERPLACE BLVD, JACKSONVILLE, 32207",30.318221,-81.653685,Southbank,Chart House Restaurant,30.318219,-81.653720,8
99,Resort,"1515 PRUDENTIAL DR, JACKSONVILLE, 32207",30.317141,-81.653036,Southbank,Lexington Hotel and Conference Center,30.317133,-81.652862,8


Let's identify the 10 most common venues in each district.

In [20]:
jax_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
jax_onehot['Zoning District'] = venues['Zoning District'] 

# move neighborhood column to the first column
fixed_columns = [jax_onehot.columns[-1]] + list(jax_onehot.columns[:-1])
jax_onehot = jax_onehot[fixed_columns]
jax_onehot.head()

,Zoning District,American Restaurant,Art Gallery,Arts & Crafts Store,Athletics & Sports,BBQ Joint,Bar,Baseball Stadium,Basketball Stadium,Beer Garden,...,Smoothie Shop,Speakeasy,Sports Bar,Steakhouse,Storage Facility,Sushi Restaurant,Tex-Mex Restaurant,Theater,Thrift / Vintage Store,Video Store
0,Brooklyn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Brooklyn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Brooklyn,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Brooklyn,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Brooklyn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Use a function to get the most common venues in each district.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
jax_onehot.shape

(101, 63)

In [23]:
jax_grouped = jax_onehot.groupby('Zoning District').mean().reset_index()
jax_grouped

,Zoning District,American Restaurant,Art Gallery,Arts & Crafts Store,Athletics & Sports,BBQ Joint,Bar,Baseball Stadium,Basketball Stadium,Beer Garden,...,Smoothie Shop,Speakeasy,Sports Bar,Steakhouse,Storage Facility,Sushi Restaurant,Tex-Mex Restaurant,Theater,Thrift / Vintage Store,Video Store
0,Brooklyn,0.000000,0.000000,0.0000,0.0,0.076923,0.000000,0.0,0.00,0.00,...,0.076923,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.000000,0.0,0.0000
1,Cathedral,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.00,0.00,...,0.000000,0.000000,0.0,0.000000,0.0625,0.0000,0.0625,0.062500,0.0,0.0625
2,Central Core,0.038462,0.038462,0.0000,0.0,0.000000,0.038462,0.0,0.00,0.00,...,0.000000,0.038462,0.0,0.038462,0.0000,0.0000,0.0000,0.038462,0.0,0.0000
3,Church,0.000000,0.000000,0.0000,0.0,0.200000,0.000000,0.0,0.00,0.00,...,0.000000,0.000000,0.0,0.000000,0.2000,0.0000,0.0000,0.000000,0.0,0.0000
4,LaVilla,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.00,0.00,...,0.000000,0.000000,0.0,0.000000,0.0000,0.0000,0.0000,0.200000,0.2,0.0000
5,Southbank,0.125000,0.000000,0.0625,0.0,0.000000,0.000000,0.0,0.00,0.00,...,0.000000,0.000000,0.0,0.062500,0.0000,0.0625,0.0000,0.000000,0.0,0.0000
6,Sports & Entertainment,0.050000,0.000000,0.0000,0.1,0.050000,0.000000,0.1,0.05,0.05,...,0.000000,0.000000,0.1,0.000000,0.0000,0.0000,0.0000,0.000000,0.0,0.0000


In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zoning District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
districts_venues_sorted = pd.DataFrame(columns=columns)
districts_venues_sorted['Zoning District'] = jax_grouped['Zoning District']

for ind in np.arange(jax_grouped.shape[0]):
    districts_venues_sorted.iloc[ind, 1:] = return_most_common_venues(jax_grouped.iloc[ind, :], num_top_venues)

districts_venues_sorted

,Zoning District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Gay Bar,Falafel Restaurant,Concert Hall,Dessert Shop,Burrito Place,Sandwich Place,Dive Bar,Smoothie Shop,Gym / Fitness Center,BBQ Joint
1,Cathedral,Coffee Shop,Record Shop,Video Store,Rental Car Location,Gift Shop,Grocery Store,Discount Store,Daycare,Indie Theater,Burrito Place
2,Central Core,Coffee Shop,Sandwich Place,Café,Harbor / Marina,Food,Food Truck,Concert Hall,Theater,Lounge,Art Gallery
3,Church,Dessert Shop,Gym,Storage Facility,BBQ Joint,Convenience Store,Video Store,Dive Bar,Deli / Bodega,Diner,Discount Store
4,LaVilla,Food Truck,Thrift / Vintage Store,Restaurant,Optical Shop,Theater,Food,Flower Shop,Flea Market,Fast Food Restaurant,Convenience Store
5,Southbank,American Restaurant,Diner,Café,Planetarium,Hotel,History Museum,Resort,Restaurant,Seafood Restaurant,Indie Movie Theater
6,Sports & Entertainment,Football Stadium,Athletics & Sports,Baseball Stadium,Sports Bar,General Entertainment,New American Restaurant,BBQ Joint,Basketball Stadium,Beer Garden,Breakfast Spot


In [101]:
#venues.to_csv('full_venues.csv')

### Results

Compare the different districts.

Foursquare found 62 businesses across 7 of the 8 downtown zoning overlays. Here are the districts ranked by most venues:

In [36]:
venue_counts = venues.groupby('Zoning District').count()['Venue'].sort_values(ascending=False)
print(venue_counts)

Zoning District
Central Core              26
Sports & Entertainment    20
Southbank                 16
Cathedral                 16
Brooklyn                  13
LaVilla                    5
Church                     5
Name: Venue, dtype: int64


In [44]:
for num in range(0,7):
    print(districts_venues_sorted['Zoning District'][num]+':')
    print('  1. '+districts_venues_sorted['1st Most Common Venue'][num])
    print('  2. '+districts_venues_sorted['2nd Most Common Venue'][num])
    print('  3. '+districts_venues_sorted['3rd Most Common Venue'][num])
    print('  4. '+districts_venues_sorted['4th Most Common Venue'][num])
    print('  5. '+districts_venues_sorted['5th Most Common Venue'][num])
    print('  6. '+districts_venues_sorted['6th Most Common Venue'][num])
    print('  7. '+districts_venues_sorted['7th Most Common Venue'][num])
    print('  8. '+districts_venues_sorted['8th Most Common Venue'][num])
    print('  9. '+districts_venues_sorted['9th Most Common Venue'][num])
    print('  10. '+districts_venues_sorted['10th Most Common Venue'][num])
    print()

Brooklyn:
  1. Gay Bar
  2. Falafel Restaurant
  3. Concert Hall
  4. Dessert Shop
  5. Burrito Place
  6. Sandwich Place
  7. Dive Bar
  8. Smoothie Shop
  9. Gym / Fitness Center
  10. BBQ Joint

Cathedral:
  1. Coffee Shop
  2. Record Shop
  3. Video Store
  4. Rental Car Location
  5. Gift Shop
  6. Grocery Store
  7. Discount Store
  8. Daycare
  9. Indie Theater
  10. Burrito Place

Central Core:
  1. Coffee Shop
  2. Sandwich Place
  3. Café
  4. Harbor / Marina
  5. Food
  6. Food Truck
  7. Concert Hall
  8. Theater
  9. Lounge
  10. Art Gallery

Church:
  1. Dessert Shop
  2. Gym
  3. Storage Facility
  4. BBQ Joint
  5. Convenience Store
  6. Video Store
  7. Dive Bar
  8. Deli / Bodega
  9. Diner
  10. Discount Store

LaVilla:
  1. Food Truck
  2. Thrift / Vintage Store
  3. Restaurant
  4. Optical Shop
  5. Theater
  6. Food
  7. Flower Shop
  8. Flea Market
  9. Fast Food Restaurant
  10. Convenience Store

Southbank:
  1. American Restaurant
  2. Diner
  3. Café
  4. Pla